In [1]:
from pathlib import Path
from ast import literal_eval
from collections import  namedtuple
from itertools import product
from tqdm import tqdm

# Essentially token types
Choice = namedtuple("Choice", "s")
Sequence = namedtuple("Sequence", "s")
Constant = namedtuple("Constant", "s")
SubRule = namedtuple("SubRule", "s")

def read_rules(text_rules, problem_number=1):
    
    patterns = {}
    unsolved = {}
    
    def make_sequence(chunk):
        "A sequence of one item is collapsed to a sub-rule"
        words = chunk.split()
        if len(words) == 1:
            sub_rule = int(words[0])
            if sub_rule in unsolved:
                return unsolved[sub_rule].copy(deep=True)
            return SubRule(int(words[0]))
        else:
            return Sequence([SubRule(int(w)) for w in words])
    
    for line in text_rules.splitlines():
        ruleid, ruledata = line.split(":")
        ruleid = int(ruleid)
        
        if '"' in ruledata:
            patterns[ruleid] = Constant(literal_eval(ruledata.strip()))
            
        elif "|" in ruledata:
            chunks = ruledata.split("|")
            subrules = [ make_sequence(chunk) for chunk in chunks ] 
            unsolved[ruleid] = Choice(subrules)
            
        else: 
            unsolved[ruleid] = make_sequence(ruledata)
            
    def resolver(item):
        if type(item) is Constant:
            return item
        elif type(item) is Sequence:
            assert len(item.s) > 1
            return Sequence([resolver(ii) for ii in item.s])
        elif type(item) is Choice:
            assert len(item.s) == 2
            return Choice([resolver(ii) for ii in item.s])
        elif type(item) is SubRule:
            return patterns[item.s]
        else:
            raise TypeError(f"What the heck??? {item} {type(item)}")
    
    while len(unsolved):
        for key, value in unsolved.copy().items():
            try:
                if type(value) is SubRule:
                    patterns[key] = resolver(patterns[value.s])
                else:
                    patterns[key] = resolver(value)
                unsolved.pop(key)
            except KeyError:
                    pass
                
    return patterns

def yield_pattern(rule):
    if type(rule) is Constant:
        yield rule.s
        
    elif type(rule) is Choice:
        yield from yield_pattern(rule.s[0])
        yield from yield_pattern(rule.s[1])
        
    elif type(rule) is Sequence:
        iterators = (yield_pattern(item) for item in rule.s)
        for parts in product(*iterators):
            yield "".join(parts)
        
    else:
        raise ValueError("Huh, interesting")

In [2]:
data = Path("rules.txt").read_text()
ruledata, imagedata = data.split("\n\n")
my_rules = read_rules(ruledata)
valid_patterns = set(yield_pattern(my_rules[0]))

match = sum([line in valid_patterns for line in imagedata.splitlines() ])
match

222

In [3]:
# Test

example_rules = '''0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"'''
example_data = '''ababbb
bababa
abbbab
aaabbb
aaaabbb'''

example_rules = read_rules(example_rules)
matching_patterns = list(yield_pattern(example_rules[0]))
print(matching_patterns) # should be 8 in the toy example

test = 0
for line in example_data.splitlines():
    for pat in matching_patterns:
        if line == pat:
            test += 1
test

['aaaabb', 'aaabab', 'abbabb', 'abbbab', 'aabaab', 'aabbbb', 'abaaab', 'ababbb']


2